In [1]:
import paho.mqtt.client as mqtt
import json
import time
import multiprocessing


In [2]:
DEVICE_NAME = "BLUE_LED"


In [3]:
# Read the config file and get the values
with open('data/config.json', 'r', encoding='utf-8') as json_data_file:
    config = json.load(json_data_file)

mqtt_host = config['mqtt']['host']
username = config['mqtt']['username']
password = config['mqtt']['password']
topic = config['mqtt']['topic']
mqttc = mqtt.Client()
mqttc.username_pw_set(username, password)
mqttc.connect(mqtt_host, 1883, 60)


0

### Change State

In [4]:
while True:

    change_state_message = {
        'origin': 'check_mqtt.ipynb',
        'messageType': 'change_state',
        'message': {
            'device_id': 'BLUE_LED',
            'state': input("Enter the state: ")
        }
    }
    match change_state_message['message']['state']:
        case 'ON':
            mqttc.publish(topic, json.dumps(change_state_message))
        case 'OFF':
            mqttc.publish(topic, json.dumps(change_state_message))
        case 'EXIT':
            print("Invalid state")
            break


Invalid state


### Check State

In [ ]:
current_state_message = {
    'origin': 'check_mqtt.ipynb',
    'messageType': 'current_state',
    'message': {
        'device_id': DEVICE_NAME,
    }
}

mqttc.publish(topic, json.dumps(current_state_message))


In [ ]:
kepp_alive_counter = 0
start_time = time.time()
def keep_alive():
    mqttc.publish(topic, json.dumps(
        {
            "origin": "check_mqtt.ipynb",
            "messageType": "keep_alive", 
            "message": {
                "keep_alive_counter": kepp_alive_counter,
                "uptime": time.time() - start_time
            }
        }
    ))
    kepp_alive_counter += 1
    time.sleep(50)

kepp_alive_process = multiprocessing.Process(target=keep_alive)
kepp_alive_process.start()
